In [1]:
# Okay, let's preprocess all data

In [2]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf

2025-07-28 10:05:04.306829: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-28 10:05:04.343736: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-28 10:05:04.343762: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-28 10:05:04.344673: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-28 10:05:04.350773: I tensorflow/core/platform/cpu_feature_guar

In [3]:
# What all do we have? We have the geenrated features, and the densities defined for all the test blocks.

In [4]:
# Do the data prep for one block, and define a function for the rest

In [5]:
# densities are here
block_0103_density_file = np.load(os.path.join("data/densities_non_overlapping/", "all_densities_block_0103.npy"))

In [6]:
block_0103_density_file.shape

(910, 20, 1)

In [7]:
# Where are the feature files?

In [8]:
# for test time points
block_0103_feature_file_test_points = np.load(os.path.join("data/predicted_sequences_from_stage_1_non_overlapping/model_1/", "pred_values_blk_0103.npy"))

In [9]:
block_0103_feature_file_test_points.shape

(910, 7, 32)

In [10]:
# for train time points
block_0103_feature_file_train_points = np.load(os.path.join("data/train_features_non_overlapping/", "train_features_block_0103.npy"))

In [11]:
block_0103_feature_file_train_points.shape

(910, 13, 32)

In [12]:
# Stack the featrue files together?

In [13]:
block_0103_features = np.hstack((block_0103_feature_file_train_points, block_0103_feature_file_test_points))

In [14]:
block_0103_features.shape

(910, 20, 32)

In [15]:
# alternative way
block_0103_features_alt = np.concatenate((block_0103_feature_file_train_points, block_0103_feature_file_test_points), axis = 1)

In [16]:
block_0103_features_alt.shape

(910, 20, 32)

In [17]:
# sanity check
np.mean(block_0103_features == block_0103_features_alt)

1.0

In [18]:
# now stack this with the response?

In [19]:
block_0103_all_data = np.concatenate((block_0103_features, block_0103_density_file), axis = 2)

In [20]:
block_0103_all_data.shape

(910, 20, 33)

In [21]:
block_0103_all_data_alt = np.dstack((block_0103_features, block_0103_density_file))

In [22]:
block_0103_all_data_alt.shape

(910, 20, 33)

In [23]:
# sanity check
np.mean(block_0103_all_data_alt == block_0103_all_data)

1.0

In [24]:
# Okay, now need to create dfs, and store so these can be later used for the BLAR model

In [25]:
block_0103_all_data_df_100 = pd.DataFrame(block_0103_all_data[100,:,:], columns = ['feature_' + str(i) for i in range(32)] + ['density'])

In [26]:
%%time
counter = 0
for row in range(block_0103_all_data.shape[0]):
    # create the df
    current_df = pd.DataFrame(block_0103_all_data[row,:,:], columns = ['feature_' + str(i) for i in range(32)] + ['density'])
    # name the df
    df_name = 'block_0103_df_' + str(counter) + '.csv'
    # df path
    df_path = "data/BLAR_ready_dfs/model_1"
    # save the df
    current_df.to_csv(os.path.join(df_path, df_name), index = False)
    counter = counter + 1

CPU times: user 1.54 s, sys: 113 ms, total: 1.66 s
Wall time: 5.56 s


In [27]:
# block_0103_all_data_df_100

In [28]:
# Seems this is what we need, but do sanity checks